In [311]:
import json
import pandas as pd
from py2neo import Graph, Node, Relationship

In [312]:
graph = Graph("http://localhost:7474/db/data/", user="neo4j", password="qwerty")

In [313]:
with open('bbox_labels_600_hierarchy.json') as file:
    data = json.load(file)

In [314]:
desc = pd.read_csv('oidv6-class-descriptions.csv')
desc.head()

,LabelName,DisplayName
0,/m/0100nhbf,Sprenger's tulip
1,/m/0104x9kv,Vinegret
2,/m/0105jzwx,Dabu-dabu
3,/m/0105ld7g,Pistachio ice cream
4,/m/0105lxy5,Woku


In [315]:
names = dict(zip(desc['LabelName'], desc['DisplayName']))

In [316]:
def getName(val):
    if val in names.keys():
        return names[val]
    else:
        return val

def dictIterate(data):
    
    node1 = Node("Label", name = getName(data['LabelName']))
    try:
        sub = data['Subcategory']
        for item in sub:
            if 'Subcategory' in item.keys():
                node2 = Node("Label", name = getName(item['LabelName']))
                relation = Relationship(node1, "PARENT_OF", node2)
                rel = Relationship.type("PARENT_OF")
                graph.merge(rel(node1, node2), 'Label', 'name')
                dictIterate(item)
            else:
                node2 = Node("Label", name = getName(item['LabelName']))
                relation = Relationship(node1, "PARENT_OF", node2)
                rel = Relationship.type("PARENT_OF")
                graph.merge(rel(node1, node2), 'Label', 'name')
    except Exception as e:
        pass

In [317]:
dictIterate(data)

### 1. Find all siblings class of a class name

In [310]:
class_name = 'Pancake'
query1 = "MATCH (a)-[r]->(b:Label {name:" + '"' + class_name + '"' + "}) RETURN a"
parent_class = graph.run(query).data()[0]['a']['name']
query2 = "MATCH (a:Label {name :" + '"' + parent_class + '"' + "})-[r]->(b) RETURN b"
siblings = graph.run(query2).data()
result = [item['b']['name'] for item in siblings]
print('Sibling Classes for', class_name, ':', ', '.join(result))

Sibling Classes for Pancake : Sushi, Dairy Product, Sandwich, Honeycomb, Vegetable, Cooking spray, Taco, Seafood, Pizza, Pasta, Mushroom, Baked goods, Egg (Food), Fruit, Guacamole, Dessert, Snack, Burrito, Pancake, Waffle, Fast food


### 2. Find Parent Class of a class name

In [307]:
class_name = 'Milk'
query1 = "MATCH (a)-[r]->(b:Label {name:" + '"' + class_name + '"' + "}) RETURN a"
parent_class = graph.run(query1).data()[0]['a']['name']
print('Parent Class of', class_name, "is :", parent_class)

Parent Class of Milk is : Dairy Product


### 3. Find All ancestors of a class name

In [306]:
class_name = 'Milk'
query1 = "MATCH p=(o:Label {name :" + '"' + class_name + '"' + "})<-[*0..6]-(a:Label) WITH *, RELATIONSHIPS(p) AS rels RETURN o, rels, a"
result = graph.run(query1).data()
print('Sibling Node :', class_name)
print('Ancestoral Nodes :', ', '.join([item.start_node['name'] for item in result[-1]['rels']]))

Sibling Node : Milk
Ancestoral Nodes : Dairy Product, Food, /m/0bl9f


### 4. Find if both class 1 and class 2 belong to the same ancestor class(es)

In [299]:
def getAncestoralClasses(query):
    query_result = graph.run(query).data()
    temp = list()
    for i in range(len(query_result)):
        temp.append([item.start_node['name'] for item in query_result[i]['rels']])
    
    result = list()
    for val in temp:
        for item in val:
            if item != '/m/0bl9f':
                result.append(item)
    
    return list(set(result))

In [305]:
class_name_1 = 'Bed'
class_name_2 = 'Table'
query1 = "MATCH p=(o:Label {name :" + '"' + class_name_1 + '"' + "})<-[*0..6]-(a:Label) WITH *, RELATIONSHIPS(p) AS rels RETURN o, rels, a"
ancestor_class_1 = getAncestoralClasses(query1)
query2 = "MATCH p=(o:Label {name :" + '"' + class_name_2 + '"' + "})<-[*0..6]-(a:Label) WITH *, RELATIONSHIPS(p) AS rels RETURN o, rels, a"
ancestor_class_2 = getAncestoralClasses(query2)
# ancestor_class_2 = [item.start_node['name'] for item in graph.run(query2).data()[-1]['rels']]

count = 0
same_ancestor_class = list()
for item in ancestor_class_1:
    if item in ancestor_class_2:
        same_ancestor_class.append(item)
        count += 1
if count > 0:
    print(class_name_1 + ' and ' + class_name_2 + ' share the same ancestoral classes')
    print('Common Ancestoral Classes :', ', '.join(same_ancestor_class))
else:
    print('No. They do not share the same ancestoral classes.')

Bed and Table share the same ancestoral classes
Common Ancestoral Classes : Furniture
